In [11]:
import pandas as pd

df = pd.read_excel('merge_all.xlsx')

In [12]:
import datetime

def extract_hour(dt):
    return dt.hour

df['시간'] = df['주문시간'].apply(extract_hour)


In [13]:
# 일자와 시간으로 그룹화
grouped = df.groupby([df['주문시간'].dt.date, '시간'])

# 상품 판매량을 각 상품마다 더해서 넣어줌
sales = grouped['상 품 명'].value_counts().unstack(fill_value=0)

# Resetting index to make '일자' and '시간' as columns
sales = sales.reset_index()

# 주문시간이라고 되어있는 열을 일자로 바꾸어줌 (시간은 따로 빼서)
sales.rename(columns={'주문시간': '일자'}, inplace=True)

In [14]:
sales.columns

Index(['일자', '시간', '.', '101Row(s)', '103Row(s)', '105Row(s)', '109Row(s)',
       '110Row(s)', '111Row(s)', '114Row(s)',
       ...
       '할인', '핫초코', '햄.치즈샌드위치', '행사) 아메리카노', '행사) 아이스아메리카노', '홍차라떼', '화이트모카',
       '화이트민트초코라떼', '휘핑크림추가', '히비스커스'],
      dtype='object', name='상 품 명', length=292)

In [15]:
import re

# '(숫자)Row(s)' 로 들어가게 된 열 제거를 위해 정규표현식 사용!
pattern = r'\d+Row\(s\)'

# 패턴에 맞춰 열 찾기
remove_pattern = [col for col in sales.columns if re.search(pattern, col)]
sales_cleaned_pattern = sales.drop(columns=remove_pattern)

sales_cleaned_pattern.head()

상 품 명,일자,시간,.,HOT아메리카노 할인,ICE아메리카노 할인,NO WHIP,가은블랜딩원두100g,가은블랜딩원두200g,고구마라떼,골드메달 (애플),...,할인,핫초코,햄.치즈샌드위치,행사) 아메리카노,행사) 아이스아메리카노,홍차라떼,화이트모카,화이트민트초코라떼,휘핑크림추가,히비스커스
0,2022-03-02,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-03-02,9,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2022-03-02,10,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2022-03-02,11,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
4,2022-03-02,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [16]:
sales_cleaned_pattern.drop(columns='.', inplace=True)

In [17]:
# 각 행의 주문 합계를 더함 (숫자형으로 맞춰줘야해서 일자와 시간은 빼고 더함)
sales_cleaned_pattern['합계'] = sales_cleaned_pattern.drop(['일자', '시간'], axis=1).sum(axis=1)
sales_cleaned_pattern.head()

상 품 명,일자,시간,HOT아메리카노 할인,ICE아메리카노 할인,NO WHIP,가은블랜딩원두100g,가은블랜딩원두200g,고구마라떼,골드메달 (애플),골드메달(스파클링),...,핫초코,햄.치즈샌드위치,행사) 아메리카노,행사) 아이스아메리카노,홍차라떼,화이트모카,화이트민트초코라떼,휘핑크림추가,히비스커스,합계
0,2022-03-02,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
1,2022-03-02,9,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,21
2,2022-03-02,10,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,21
3,2022-03-02,11,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,62
4,2022-03-02,12,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,47


In [18]:
print(list(sales_cleaned_pattern.columns))

['일자', '시간', 'HOT아메리카노 할인', 'ICE아메리카노 할인', 'NO WHIP', '가은블랜딩원두100g', '가은블랜딩원두200g', '고구마라떼', '골드메달    (애플)', '골드메달(스파클링)', '녹차라떼', '덜달게', '딸기라떼', '딸기스무디', '딸기연유라떼 14oz', '레모니카노', '레몬글라스', '레몬에이드', '레몬차', '레몬차 ICE', '많이(얼음/물/우유)', '망고스무디', '망고에이드', '민트초코', '바닐라라떼', '복숭아아이스티', '분다버그      (라임, 자몽)', '블루베리라떼', '블루베리스무디', '사이즈업 중간잔 (AME)', '사이즈업 큰잔', '상품권', '생강차', '생강차 ICE', '샷추가', '석류아이스티', '시럽추가', '아메리카노', '아이리쉬라떼', '아이스 레몬글라스', '아이스 얼그레이홍차', '아이스 잉글리쉬 블랙퍼스트', '아이스 캐모마일', '아이스 페퍼민트', '아이스 히비스커스', '아이스고구마라떼', '아이스녹차라떼', '아이스딸기라떼', '아이스레모니카노', '아이스민트초코(휘핑)', '아이스바닐라라떼', '아이스블루베리라떼', '아이스아메리카노', '아이스아이리쉬라떼', '아이스연유라떼', '아이스오곡라떼', '아이스초코(휘핑)', '아이스카라멜마끼아또', '아이스카페라떼', '아이스카페모카', '아이스토피넛라떼', '아이스토피넛모카', '아이스홍차라떼', '아이스화이트모카', '아이스화이트민트초코라떼', '아포가토', '얼그레이홍차', '얼음컵', '에스프레소', '연유라떼', '오곡라떼', '오렌지에이드', '우유', '유자스무디', '유자에이드', '유자차', '유자차 ICE', '잉글리쉬 블랙퍼스트', '자몽에이드', '자몽차', '자몽차 ICE', '적게(얼음/물/우유)', '젤라또', '직무박람회할인', '천혜향스무디', '천혜향에이드', '천혜향차', '천혜향차 ICE', '청포도에이드', '카라멜마끼아또', '카야샌드위치', '카페라떼', '

In [19]:
sales_cleaned_pattern.to_excel("check.xlsx", index=False)